In [ ]:
# 我们需要对Broker类进行进一步的改进，以允许对每个标的进行买入（做多）和卖出（做空）操作。
# 同时，我们也需要记录每个持仓的买卖类型（多头或空头），并在计算市值时对空头持仓进行相应的处理。

In [ ]:
class Broker:
    def __init__(self, cash: float):
        self.cash = cash
        self.positions = {}
        self.position_values = {}
        self.trades = []

    def buy(self, symbol, price, quantity):
        # 做多一定数量的标的
        self._process_order(symbol, price, quantity)

    def sell(self, symbol, price, quantity):
        # 做空一定数量的标的
        self._process_order(symbol, price, -quantity)  # 用负数表示做空

    # 把订单处理全部交给这个函数
    def _process_order(self, symbol, price, quantity):
        # 如果要处理的标的没有持仓，则先创建持仓
        if symbol not in self.positions:
            self.positions[symbol] = 0
        # 否则，更新持仓数量
        self.positions[symbol] += quantity

        cost = price * abs(quantity)

        if quantity > 0:  # 做多
            if self.cash >= cost: # 钱够
                self.cash -= cost
                self.trades.append((symbol, 'BUY', quantity, price))
        else:  # 做空
            self.cash += cost  # 假设保证金等机制，先获得卖出收益（可以类比多头仓位平仓）
            self.trades.append((symbol, 'SELL', quantity, price))
    
    def update_position_value(self, symbol, price):
        if symbol in self.positions:
            # 做空时，市值的计算需要使用初始卖出时的价格减去当前价格的差值
            position = self.positions[symbol]
            if position > 0:  # 多头持仓
                self.position_values[symbol] = position * price
            else:  # 空头持仓
                # 假设空头持仓时记录了一个初始卖出价格，这里简化处理使用最后一次交易价格
                last_sell_price = next(trade[3] for trade in reversed(self.trades) if trade[0] == symbol)
                self.position_values[symbol] = abs(position) * (last_sell_price - price)

    def result(self):
        total_position_value = sum(self.position_values.values())
        return self.cash + total_position_value